In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
Fs = 16000
sr = 16000
n_fft = 1024
n_mfcc = 64
hop_length = 512
win_length = 1024

In [2]:
%cd /content/drive/MyDrive/Spectrograms_xx

/content/drive/MyDrive/Spectrograms_xx


In [3]:
import numpy as np
import librosa
import os

In [4]:

def stft_to_mfcc(x,Fs,n_mfcc):
    x_mfcc = []
    xt = x.T
    for i in range(0,312,4):
        mfcc = librosa.feature.mfcc(S = xt[:][i:i+4].T, sr = Fs, n_mfcc = n_mfcc)
        if(len(mfcc[0])*(len(mfcc))==64*4):
            x_mfcc.append(mfcc)
        # if(len(x_mfcc[0])==78):break
    return np.array(x_mfcc)

In [5]:
X_music=[]
for i in range(1,90):
  if i!=19:
    x = np.load('Music/music_{num}_.npy'.format(num=i))
    x_mfcc = stft_to_mfcc(x,Fs,n_mfcc)
    for j in range(x_mfcc.shape[0]):
      X_music.append(x_mfcc[j])



X_music = np.array(X_music)

In [6]:
X_music.shape

(6848, 64, 4)

In [ ]:
X_speech=[]
for i in range(1,90):
  x = np.load('Speech/speech_{num}_.npy'.format(num=i))
  x_mfcc = stft_to_mfcc(x,Fs,n_mfcc)
  for j in range(x_mfcc.shape[0]):
    X_speech.append(x_mfcc[j])
  
X_speech = np.array(X_speech)

In [9]:
X_speech.shape

(6818, 64, 4)

In [10]:
X_silence=[]
for i in range(1,51):
  if i<10:
    x = np.load('Silence/silence_0{num}_.npy'.format(num=i))
  else:
    x = np.load('Silence/silence_{num}_.npy'.format(num=i))
  x_mfcc = stft_to_mfcc(x,Fs,n_mfcc)
  for j in range(x_mfcc.shape[0]):
    X_silence.append(x_mfcc[j])

X_silence = np.array(X_silence)

In [11]:
X_silence.shape

(3900, 64, 4)

In [12]:
X = []
Y = []

for i in range(X_music.shape[0]):
  X.append(X_music[i])
  Y.append([1,0,0])

for i in range(X_speech.shape[0]):
  X.append(X_speech[i])
  Y.append([0,1,0])

for i in range(X_silence.shape[0]):
  X.append(X_silence[i])
  Y.append([0,0,1])


X = np.array(X)
Y = np.array(Y)

(17566, 3)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
 X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=3)

In [15]:
X_train = X_train.reshape([X_train.shape[0],256])
X_train.shape

(14931, 256)

In [16]:
X_val = X_val.reshape([X_val.shape[0],256])
X_val.shape

(2635, 256)

In [17]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,LSTM,TimeDistributed
from keras.layers import Convolution2D, MaxPooling2D,MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import Adam,SGD
from keras.utils import np_utils
from sklearn import metrics
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
import datetime
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [18]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape=(256,)))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                16448     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 3)                 5

In [19]:
model.compile(optimizer='adam',
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=20,batch_size=12,validation_data=(X_val,Y_val))

Epoch 1/20
1245/1245 [==============================] - 4s 3ms/step - loss: 1.2362 - accuracy: 0.7983 - val_loss: 0.1665 - val_accuracy: 0.9366
Epoch 2/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.2281 - accuracy: 0.9265 - val_loss: 0.1259 - val_accuracy: 0.9560
Epoch 3/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.1634 - accuracy: 0.9463 - val_loss: 0.0816 - val_accuracy: 0.9742
Epoch 4/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.1182 - accuracy: 0.9618 - val_loss: 0.1365 - val_accuracy: 0.9545
Epoch 5/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.1084 - accuracy: 0.9666 - val_loss: 0.0579 - val_accuracy: 0.9852
Epoch 6/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.0752 - accuracy: 0.9765 - val_loss: 0.0568 - val_accuracy: 0.9852
Epoch 7/20
1245/1245 [==============================] - 3s 2ms/step - loss: 0.0679 - accuracy: 0.9792 - val_loss: 0.0559 - val_accuracy:

In [20]:
tf.keras.models.save_model(model,'nn_corrected.h5')